In [1]:
!python main.py

value and its type
0 : <class 'int'>
value and its type
AMZN : <class 'str'>
value and its type
2023-10-06 : <class 'datetime.date'>
inside timestamp
value and its type
01:00PM : <class 'str'>
value and its type
How I Plan to Save $100 on Halloween This Year This writer is planning for a budget Halloween this year. Read on to learn how she's getting it done. : <class 'str'>
value and its type
1 : <class 'int'>
value and its type
AMZN : <class 'str'>
value and its type
2023-10-06 : <class 'datetime.date'>
inside timestamp
value and its type
12:28PM : <class 'str'>
value and its type
Amazon Launches Its First Satellites Into Space Today. Heres How to Watch. The company plans to send more than 3,000 pieces of hardware into low Earth orbit over the next six years for its space Wi-Fi business. : <class 'str'>
value and its type
2 : <class 'int'>
value and its type
AMZN : <class 'str'>
value and its type
2023-10-06 : <class 'datetime.date'>
inside timestamp
value and its type
12:25PM : <clas

c:\Users\user\OneDrive\Documents\SE_Factory\FSD\Final_Project\us_recession_analysis\database_handler.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)


In [ ]:
from prehook import execute_prehook
execute_prehook()

(['finviz'],
 ['financial_news'],
 [Empty DataFrame
  Columns: [ticker, date, time, text]
  Index: []])

In [ ]:
from lookups import DateField,WebScrapeDFTemplate

WebScrapeDFTemplate.__name__

'WebScrapeDFTemplate'

In [ ]:
from webscrape import scrape_website
from datetime import datetime

In [ ]:
news_df = scrape_website(etl_date= datetime(2023, 10, 6))

In [ ]:
news_df

,news_id,ticker,date,time,text
0,0,AMZN,2023-10-06,10:20AM,Amazon leaked documents show an exciting chang...
1,1,AMZN,2023-10-06,09:53AM,2 Stock-Split Artificial Intelligence (AI) Sto...
2,2,AMZN,2023-10-06,09:15AM,"Hollywood strikes have cost the US economy 45,..."
3,3,AMZN,2023-10-06,09:00AM,"Is Trending Stock Amazon.com, Inc. (AMZN) a Bu..."
4,4,AMZN,2023-10-06,08:15AM,Shift4 and Amazon Team Up for Checkout-Free Sh...
5,5,AMZN,2023-10-06,08:00AM,New This Holiday Season: Discounts on Shipping...
6,6,AMZN,2023-10-06,08:00AM,3 Top Cloud Stocks to Buy in October The cloud...
7,7,AMZN,2023-10-06,07:10AM,Etsy Still Manages to Outshine Amazon in This ...
8,8,AMZN,2023-10-06,06:05AM,Amazon Just Found a Smart Way to Add Billions ...
9,9,AMZN,2023-10-06,05:50AM,Microsoft Invested in OpenAI. Amazon and Googl...
